In [2]:
import numpy as np
import cupy
import networkx as nx
from numba import cuda
import warnings
from numba.core.errors import NumbaPerformanceWarning


warnings.simplefilter('ignore', category=NumbaPerformanceWarning)



In [112]:

def _add_node(i : int, source : np.ndarray, temp : np.ndarray, subgraph : np.ndarray):
    '''
    Iteratively extract induced subgraph containing 'i' from source in subgraph. temp is used as a temporary location.
    O(n) time complexity
    O(2*s) space complexity
    '''
    # Copy i-th row in the temp matrix
    temp[i,:] = source[i, :]
    # Self-and to remove other edges
    row =  np.bitwise_and(temp[:,i], temp[i,:].T)
    # # Copy relevant row to target graph
    subgraph[:,i] = row
    subgraph[i,:] = row.T 
    #subgraph[:,:] = np.bitwise_and(temp, temp.T)

def induced_subgaph(graph, nodes):
    iab_g = np.zeros_like(graph)
    temp_g =  np.zeros_like(graph)
    for n in nodes:
        _add_node(n, graph, temp_g, iab_g)
    return iab_g


#Generate random graph
g = nx.gnp_random_graph(5, 0.4)
#Convert to np adjmat
adjmat = nx.to_numpy_matrix(g).astype(np.uint32)
#Extract subgraph with custom method
subg_np = induced_subgaph(adjmat, [1,2,3,4])
print(subg_np)
print(nx.Graph(subg_np).edges() == nx.subgraph(g, [1,2,3,4]).edges())


[[0 0 0 0 0]
 [0 0 0 0 1]
 [0 0 0 0 1]
 [0 0 0 0 0]
 [0 1 1 0 0]]
True


In [117]:

def cu_add_node(i : int, source, temp, subgraph):
    copy_row[source.shape[0], 1](i, source, temp)
    bitwise_rowcol[source.shape[0], 1](i, temp, subgraph)

@cuda.jit()
def copy_row(i, source, dest):
    j = cuda.grid(1)
    if j<source.shape[0]:
        dest[i,j] = source[i,j]

@cuda.jit()
def bitwise_rowcol(i, source, dest):
    j = cuda.grid(1)
    if j<source.shape[0]:
        dest[i,j] = dest[j, i] = source[i,j] & source[j,i]




def induced_subgraph(graph, nodes):
    c_graph = cuda.to_device(graph)
    c_subgraph = cuda.to_device(np.zeros_like(adjmat))
    c_temp = cuda.to_device(np.zeros_like(adjmat))
    for n in nodes:
        cu_add_node(n, c_graph, c_temp, c_subgraph)
    return c_subgraph

subg = induced_subgraph(adjmat, [1,2,3,4])

cupy.linalg.eigh(cupy.asarray(subg))

/home/gabriele.gemmi/.virtualenvs/truenets/lib/python3.8/site-packages/numba/cuda/dispatcher.py:488: NumbaPerformanceWarning: Grid size 5 will likely result in GPU under-utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))
/home/gabriele.gemmi/.virtualenvs/truenets/lib/python3.8/site-packages/numba/cuda/dispatcher.py:488: NumbaPerformanceWarning: Grid size 5 will likely result in GPU under-utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))


(array([-1.41421356e+00,  0.00000000e+00,  0.00000000e+00,  7.54604712e-17,
         1.41421356e+00]),
 array([[ 0.00000000e+00,  1.00000000e+00,  0.00000000e+00,
          0.00000000e+00,  0.00000000e+00],
        [-5.00000000e-01,  0.00000000e+00,  0.00000000e+00,
          7.07106781e-01, -5.00000000e-01],
        [-5.00000000e-01,  0.00000000e+00,  0.00000000e+00,
         -7.07106781e-01, -5.00000000e-01],
        [ 0.00000000e+00,  0.00000000e+00, -1.00000000e+00,
          0.00000000e+00,  0.00000000e+00],
        [ 7.07106781e-01,  0.00000000e+00,  0.00000000e+00,
          1.11022302e-16, -7.07106781e-01]]))

Multi matrix version

In [34]:

g = nx.Graph()
g.add_edge(1,2)
g.add_edge(3,4)
g.add_edge(5,6)
g.add_edge(7,8)
g2 = nx.Graph()
g2.add_edge(1,2)
g2.add_edge(2,3)
g2.add_edge(3,4)
g2.add_edge(4,5)
g2.add_edge(5,6)
g2.add_edge(6,7)
g2.add_edge(7,8)
#Convert to np adjmat
adjmat = nx.to_numpy_matrix(g).astype(np.uint32)
adjmat2 = nx.to_numpy_matrix(g2).astype(np.uint32)
#get laplacian matrix

l1 = nx.laplacian_matrix(g).toarray()
l2 = nx.laplacian_matrix(g2).toarray()
print(l1)
print(l2)
print("Eigenval of m1")
print(cupy.linalg.eigvalsh(cupy.asarray(l1)))
print("Eigenval of m2")
print(cupy.linalg.eigvalsh(cupy.asarray(l2)))
m = np.zeros(shape=(2, adjmat.shape[0], adjmat.shape[1]))
m[0] = l1
m[1] = l2

print(m)

print("Combined")
print(cupy.linalg.eigvalsh(cupy.asarray([l1,l2])))



[[ 1 -1  0  0  0  0  0  0]
 [-1  1  0  0  0  0  0  0]
 [ 0  0  1 -1  0  0  0  0]
 [ 0  0 -1  1  0  0  0  0]
 [ 0  0  0  0  1 -1  0  0]
 [ 0  0  0  0 -1  1  0  0]
 [ 0  0  0  0  0  0  1 -1]
 [ 0  0  0  0  0  0 -1  1]]
[[ 1 -1  0  0  0  0  0  0]
 [-1  2 -1  0  0  0  0  0]
 [ 0 -1  2 -1  0  0  0  0]
 [ 0  0 -1  2 -1  0  0  0]
 [ 0  0  0 -1  2 -1  0  0]
 [ 0  0  0  0 -1  2 -1  0]
 [ 0  0  0  0  0 -1  2 -1]
 [ 0  0  0  0  0  0 -1  1]]
Eigenval of m1
[2.22044605e-16 2.22044605e-16 2.22044605e-16 2.22044605e-16
 2.00000000e+00 2.00000000e+00 2.00000000e+00 2.00000000e+00]
Eigenval of m2
[-1.34744607e-16  1.52240935e-01  5.85786438e-01  1.23463314e+00
  2.00000000e+00  2.76536686e+00  3.41421356e+00  3.84775907e+00]
[[[ 1. -1.  0.  0.  0.  0.  0.  0.]
  [-1.  1.  0.  0.  0.  0.  0.  0.]
  [ 0.  0.  1. -1.  0.  0.  0.  0.]
  [ 0.  0. -1.  1.  0.  0.  0.  0.]
  [ 0.  0.  0.  0.  1. -1.  0.  0.]
  [ 0.  0.  0.  0. -1.  1.  0.  0.]
  [ 0.  0.  0.  0.  0.  0.  1. -1.]
  [ 0.  0.  0.  0.  0.  0. -1.

<class 'networkx.utils.decorators.argmap'> compilation 16:4: FutureWarning: laplacian_matrix will return a scipy.sparse array instead of a matrix in Networkx 3.0.


In [9]:
#graph = nx.gnp_random_graph(n=5, p=1)
graph = nx.gnp_random_graph(555, 0.4)
adjmat = nx.to_numpy_matrix(graph).astype(np.uint8)


In [13]:
# def cu_add_node(i : int, source, temp, subgraph):
#     copy_row[source.shape[0], 1](i, source, temp)
#     bitwise_rowcol[source.shape[0], 1](i, temp, subgraph)



@cuda.jit()
def copy_row(source, dest):
    i,j = cuda.grid(2)
    if j<source.shape[0] and i<source.shape[0]:
        dest[i,i,j] = source[i,j]

@cuda.jit()
def bitwise_rowcol(source, dest):
    i,j = cuda.grid(2)
    if j<source.shape[0] and i<source.shape[0]:
        dest[i,i,j] = dest[i, j, i] = source[i, i, j] & source[i, j, i] 

def cu_add_all_node(source, temp, subgraph):
    copy_row[source.shape, (1,1)](source, temp)
    bitwise_rowcol[source.shape, (1,1)](temp, subgraph)

def cu_connectivity(subgraph, laplacian):
    cu_laplacian[subgraph.shape, (1,1)](subgraph, laplacian)
    eigv = cupy.linalg.eigvalsh(cupy.asarray(laplacian))
    cc = np.zeros(shape=subgraph.shape[0])
    print(subgraph.shape)
    for i in range(subgraph.shape[0]):
        for j in range(subgraph.shape[1]):
            if eigv[i,j] > 1e-10:
                cc[i] = j
                break
    print(eigv)
    return cc
        
    

@cuda.jit()
def cu_laplacian(graph, lapl):
    i,j = cuda.grid(2) #i is the graph index, j is the row index
    if j<graph.shape[0] and i<graph.shape[0]:
        d = 0
        for k in range(graph.shape[0]): #iterate on the row
            d+=graph[i, j, k]
            lapl[i,j,k] = -1*graph[i,j,k] #set the negative values
        lapl[i, j, j] = d  #Set the diagonal equal to the degree



@cuda.jit()
def copy_mat(m_i, mat1, mat2):
    i,j = cuda.grid(2)
    if j<mat1.shape[1] and i<mat1.shape[2]:
        for k in range(mat1.shape[0]):
            if k != m_i:
                mat1[k,i,j] = mat1[m_i, i, j]
                mat2[k,i,j] = mat2[m_i, i, j]



def test(graph):
    assert(graph.shape[0] == graph.shape[1])
    n = graph.shape[0]
    c_graph = cuda.to_device(graph)
    c_subgraph = cuda.to_device(np.zeros(shape=(n,n,n), dtype=np.uint8))
    c_temp = cuda.to_device(np.zeros(shape=(n,n,n), dtype=np.uint8))
    c_lapl = cuda.to_device(np.zeros(shape=(n,n,n), dtype=np.int32))

    
    cu_add_all_node(c_graph, c_temp, c_subgraph)
    copy_mat[graph.shape, (1,1)](i, c_temp, c_subgraph)
    eig = cu_connectivity(c_subgraph, c_lapl)
    
#test(adjmat)

In [16]:
n = adjmat.shape[0]
c_graph = cuda.to_device(adjmat)
c_subgraph = cuda.to_device(np.zeros(shape=(n,n,n), dtype=np.uint8))
c_temp = cuda.to_device(np.zeros(shape=(n,n,n), dtype=np.uint8))
c_lapl = cuda.to_device(np.zeros(shape=(n,n,n), dtype=np.int32))


cu_add_all_node(c_graph, c_temp, c_subgraph)
copy_mat[c_graph.shape, (1,1)](0, c_temp, c_subgraph)



In [22]:
%time cu_laplacian[c_subgraph.shape, (1,1)](c_subgraph, c_lapl)


CPU times: user 447 µs, sys: 15 µs, total: 462 µs
Wall time: 411 µs


In [23]:
%time culap = cupy.asarray(c_lapl)

CPU times: user 101 µs, sys: 3 µs, total: 104 µs
Wall time: 108 µs


In [26]:
%time eigv = cupy.linalg.eigvalsh(culap)

CPU times: user 681 ms, sys: 3.78 ms, total: 685 ms
Wall time: 684 ms
